In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

def region_of_interest(img, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


# highlight edges using canny
def highlight_edges(image):
    # mask white and yellow color
    filtered = filter_white_yellow(image)
    
    # blur input
    blurred = cv2.GaussianBlur(filtered, (5, 5), 0)

    # convert to grayscale
    gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)   

    # highlight edges
    return cv2.Canny(gray, 100, 200)


# calculate a viewport
def calculate_viewport(width, height, side):
    # camera-specific parameters!
    field_depth = int(height * 0.4)
    field_width_near = int(width * 0.85)
    field_width_far = int(width * 0.08)

    if (side == "left"):
        region = np.array([[[(width - field_width_near) / 2, height - 1],
                            [(width - field_width_far) / 2, height - field_depth],
                            [width / 2, height - field_depth],
                            [width / 2, height - 1]]], dtype=np.int32)
    elif (side == "right"):
        region = np.array([[[width / 2, height - 1],
                            [width / 2, height - field_depth],
                            [width - (width - field_width_far) / 2, height - field_depth],
                            [width - (width - field_width_near) / 2, height - 1]]],
                          dtype=np.int32)
    else:
        region = np.array([[[(width - field_width_near) / 2, height - 1],
                            [(width - field_width_far) / 2, height - field_depth],
                            [width / 2, height - field_depth],
                            [width - (width - field_width_far) / 2, height - field_depth],
                            [width - (width - field_width_near) / 2, height - 1]]],
                          dtype=np.int32)

    return region


# extract lines from an canny-edge-image using hough + mask
def extract_lines(image, side):
    height = image.shape[0]
    width = image.shape[1]

    viewport = calculate_viewport(width, height, side)
    masked_edges = region_of_interest(image, viewport)

    threshold = 9     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 12 #minimum number of pixels making up a line
    max_line_gap = 5    # maximum gap in pixels between connectable line segments
    return cv2.HoughLinesP(masked_edges, 1, np.pi/180, threshold, np.array([]), min_line_length, max_line_gap)


# render all lines into the image
def draw_lines(img, lines, color=[255, 0, 0], thickness = 10):
    if lines is None:
        return

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)


def draw_viewport(img, color=[0, 0, 255], thickness = 2):
    viewport = calculate_viewport(img.shape[1], img.shape[0], "complete")
    for i in range(0, 4):
        cv2.line(img,
                 (viewport[0, i, 0], viewport[0, i, 1]),
                 (viewport[0, i+1, 0], viewport[0, i+1, 1]), color, thickness)


# drop lines that have a low slope (parallel to camera)
def slope_filter(lines):
    result = []
    if lines is not None:
        for line in lines:
            for x1,y1,x2,y2 in line:
                if (abs(y2 - y1) == 0):
                    slope = 0
                else:
                    slope = abs(x2 - x1) / abs(y2 - y1)
                # only keep lines that are more in the direction south/north
                # than east/west (slope >45 degrees)
                if (slope > 1.0):
                    result.append(line)

    return result


# find best matching 1d-poly and draw on full width
def approximate_line(lines, image_width, enable_slope_filter=True):
    if lines is None:
        return [];
    
    if enable_slope_filter:
        lines = slope_filter(lines)

    x = []
    y = []
    for line in lines:
        for x1, y1, x2, y2 in line:
            x.append(x1)
            y.append(y1)
            x.append(x2)
            y.append(y2)

    if (not x or not y):
        return []

    fit = np.poly1d(np.polyfit(x, y, 1))
    return [[[0, int(fit(0)), image_width, int(fit(image_width))]]]


def filter_white_yellow(image):
    hue = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)   

    lower_yellow = np.array([20, 50, 50])
    upper_yellow = np.array([40, 255, 255])
    mask_yellow = cv2.inRange(hue, lower_yellow, upper_yellow)

    lower_white = np.array([200, 200, 200])
    upper_white = np.array([255, 255, 255])
    mask_white = cv2.inRange(image, lower_white, upper_white)

    # mask white and yellow-ish areas in original image
    mask = mask_yellow + mask_white
    res = cv2.bitwise_and(image, image, mask=mask)
    return res


def process_image(image):
    edges = highlight_edges(image)
    lines_left = extract_lines(edges, "left")
    lines_right = extract_lines(edges, "right")
    
    # find optimal lines (one per side)
    left = approximate_line(lines_left, image.shape[1])
    right = approximate_line(lines_right, image.shape[1])

    # render optimized lines
    line_image = np.copy(image)*0
    draw_lines(line_image, left)
    draw_lines(line_image, right)
    # cut lines on below horizon
    line_image = region_of_interest(line_image,
                                    calculate_viewport(image.shape[1], image.shape[0], "complete"))
                                    
    # enable to add raw detected lines + viewport to output
    if False:
        raw_line_image = np.copy(line_image)*0
        draw_lines(raw_line_image, lines_left, [0,255,0], 3)
        draw_lines(raw_line_image, lines_right, [0,255,0], 3)
        draw_viewport(raw_line_image)
        # blend into rendering of optimized lines
        line_image = cv2.addWeighted(raw_line_image, 0.8, line_image, 1, 0)

    return cv2.addWeighted(image, 1, line_image, 0.5, 0)

In [2]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="        {0}">
</video>
""".format(white_output))

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:06<00:00, 33.87it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 5.75 s, sys: 513 ms, total: 6.27 s
Wall time: 6.78 s


In [3]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:20<00:00, 33.49it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 19 s, sys: 783 ms, total: 19.8 s
Wall time: 21.1 s


## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [4]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:17<00:00, 14.64it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 13.3 s, sys: 903 ms, total: 14.2 s
Wall time: 19.4 s
